In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# importing file from gdrive
link = 'https://drive.google.com/open?id=1N-NiW842fQtA7-aSWOaVOufxTRAbBgTZ' # Paste the shareable link here

fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') # file will be stored as train.csv in the colab directory

1N-NiW842fQtA7-aSWOaVOufxTRAbBgTZ


In [0]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
# reading data
raw_data = pd.read_csv('train.csv', encoding = 'unicode_escape', parse_dates=['InvoiceDate'])
raw_data = raw_data[pd.notnull(raw_data['CustomerID'])]
raw_data['InvoiceDate'] = raw_data['InvoiceDate'].dt.date
raw_data['CustomerID'] = raw_data['CustomerID'].astype('int')
raw_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom


some of the descriptions have spaces that need to be removed. We’ll also drop the rows that don’t have invoice numbers and remove the credit transactions (those with invoice numbers containing C)

In [0]:
raw_data['Description'] = raw_data['Description'].str.strip()
raw_data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
raw_data['InvoiceNo'] = raw_data['InvoiceNo'].astype('str')
raw_data = raw_data[~raw_data['InvoiceNo'].str.contains('C')]

In [0]:
raw_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom


In [0]:
basket = (raw_data.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [0]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
#basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [0]:
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,200 RED + WHITE BENDY STRAWS,3 BIRDS CANVAS SCREEN,3 BLACK CATS W HEARTS BLANK CARD,3 DRAWER ANTIQUE WHITE WOOD CABINET,3 GARDENIA MORRIS BOXED CANDLES,3 HEARTS HANGING DECORATION RUSTIC,3 HOOK HANGER MAGIC GARDEN,3 HOOK PHOTO SHELF ANTIQUE WHITE,3 PIECE SPACEBOY COOKIE CUTTER SET,3 PINK HEN+CHICKS IN BASKET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 RAFFIA RIBBONS VINTAGE CHRISTMAS,...,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#0.005 gave good results
frequent_itemsets = apriori(basket_sets, min_support=0.009, use_colnames=True)

In [0]:
rules = association_rules(frequent_itemsets, metric="lift")
rules.sort_values('confidence', ascending = False, inplace = True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
212,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010035,0.010250,0.009495,0.946237,92.312847,0.009392,18.409344
1351,"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE PINK)",(REGENCY TEA PLATE GREEN),0.010628,0.014566,0.009981,0.939086,64.470013,0.009826,16.177537
208,(HERB MARKER PARSLEY),(HERB MARKER ROSEMARY),0.010088,0.010250,0.009387,0.930481,90.775795,0.009284,14.237168
213,(HERB MARKER ROSEMARY),(HERB MARKER THYME),0.010250,0.010035,0.009495,0.926316,92.312847,0.009392,13.435246
1358,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.010358,0.017533,0.009549,0.921875,52.578077,0.009367,12.575572
...,...,...,...,...,...,...,...,...,...
788,(WHITE HANGING HEART T-LIGHT HOLDER),(ZINC METAL HEART DECORATION),0.106334,0.025680,0.009387,0.088280,3.437729,0.006657,1.068662
770,(WHITE HANGING HEART T-LIGHT HOLDER),(SMALL WHITE HEART OF WICKER),0.106334,0.031398,0.009387,0.088280,2.811614,0.006048,1.062389
782,(WHITE HANGING HEART T-LIGHT HOLDER),(WOOD 2 DRAWER CABINET WHITE FINISH),0.106334,0.027460,0.009171,0.086251,3.140946,0.006251,1.064340
772,(WHITE HANGING HEART T-LIGHT HOLDER),(SPOTTY BUNTING),0.106334,0.054111,0.009171,0.086251,1.593960,0.003418,1.035173


In [0]:
rules[ (rules['lift'] >= 5) &
       (rules['confidence'] >= 0.5) ]

NameError: ignored

In [0]:
rules.to_csv('market_basket.csv')

In [0]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
212,(HERB MARKER THYME),(HERB MARKER ROSEMARY),0.010035,0.010250,0.009495,0.946237,92.312847,0.009392,18.409344
1351,"(REGENCY TEA PLATE ROSES, REGENCY TEA PLATE PINK)",(REGENCY TEA PLATE GREEN),0.010628,0.014566,0.009981,0.939086,64.470013,0.009826,16.177537
208,(HERB MARKER PARSLEY),(HERB MARKER ROSEMARY),0.010088,0.010250,0.009387,0.930481,90.775795,0.009284,14.237168
213,(HERB MARKER ROSEMARY),(HERB MARKER THYME),0.010250,0.010035,0.009495,0.926316,92.312847,0.009392,13.435246
1358,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.010358,0.017533,0.009549,0.921875,52.578077,0.009367,12.575572
...,...,...,...,...,...,...,...,...,...
788,(WHITE HANGING HEART T-LIGHT HOLDER),(ZINC METAL HEART DECORATION),0.106334,0.025680,0.009387,0.088280,3.437729,0.006657,1.068662
770,(WHITE HANGING HEART T-LIGHT HOLDER),(SMALL WHITE HEART OF WICKER),0.106334,0.031398,0.009387,0.088280,2.811614,0.006048,1.062389
782,(WHITE HANGING HEART T-LIGHT HOLDER),(WOOD 2 DRAWER CABINET WHITE FINISH),0.106334,0.027460,0.009171,0.086251,3.140946,0.006251,1.064340
772,(WHITE HANGING HEART T-LIGHT HOLDER),(SPOTTY BUNTING),0.106334,0.054111,0.009171,0.086251,1.593960,0.003418,1.035173


In [0]:
# !pip install apyori
# import apyori as apriori

In [0]:
# association_rules = apriori(basket_sets,min_suport = 0.50, min_confidence = 0.1, min_lift = 1.2, min_length = 2)
# association_results = list(association_rules)

In [0]:
# frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
# frequent_itemsets

In [0]:
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0)
# rules.head()